# Training Models

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from gensim.models import Word2Vec
import numpy as np

# Hyperparameters
maxlen = 100  # Maximum sequence length
embedding_dim = 100  # Dimension of word embeddings
lstm_units = 128  # Number of LSTM units (increased for more complexity)
batch_size = 32  # Batch size for training
epochs = 5  # Number of training epochs

# Load the dataset
Reg_data = pd.read_csv("training_dataset.csv")

X_body = Reg_data['Body_tokens'].apply(lambda x: eval(x)).values
X_title = Reg_data['Title_tokens'].apply(lambda x: eval(x)).values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_body, X_title, test_size=0.2, random_state=42)

# Initialize Tokenizer and fit on the combined text
Reg_tokenizer = Tokenizer()
Reg_tokenizer.fit_on_texts(X_body)
Reg_tokenizer.fit_on_texts(X_title)

# Convert tokens to sequences of integers
X_train_seq = Reg_tokenizer.texts_to_sequences(X_train)
X_test_seq = Reg_tokenizer.texts_to_sequences(X_test)
y_train_seq = Reg_tokenizer.texts_to_sequences(y_train)
y_test_seq = Reg_tokenizer.texts_to_sequences(y_test)

# Pad sequences to make them of equal length
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen, padding='post', truncating='post')
y_train_pad = pad_sequences(y_train_seq, maxlen=maxlen, padding='post', truncating='post')
y_test_pad = pad_sequences(y_test_seq, maxlen=maxlen, padding='post', truncating='post')

# Train the Word2Vec model
word2vec_model = Word2Vec(sentences=X_body + X_title, vector_size=embedding_dim, window=5, min_count=1, workers=4)

# Initialize the embedding matrix with zeros
embedding_matrix = np.zeros((len(Reg_tokenizer.word_index) + 1, embedding_dim))

# Fill the embedding matrix with pretrained word vectors
for word, i in Reg_tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

# Define the LSTM model architecture with pretrained embeddings
Reg_model = Sequential()
Reg_model.add(Embedding(input_dim=len(Reg_tokenizer.word_index) + 1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
Reg_model.add(LSTM(units=lstm_units, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
Reg_model.add(Dense(len(Reg_tokenizer.word_index) + 1, activation='softmax'))  # Predicting word indices

# Compile the model
Reg_model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

# Train the model
Reg_model.fit(X_train_pad, y_train_pad, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# Evaluate the model
loss, accuracy = Reg_model.evaluate(X_test_pad, y_test_pad)
print("Test Accuracy:", accuracy)



Epoch 1/5


1156/1156 [==============================] - 3215s 3s/step - loss: 0.7960 - accuracy: 0.9383 - val_loss: 0.4808 - val_accuracy: 0.9402
Epoch 2/5
1156/1156 [==============================] - 18258s 16s/step - loss: 0.4676 - accuracy: 0.9404 - val_loss: 0.4697 - val_accuracy: 0.9405
Epoch 3/5
1156/1156 [==============================] - 3084s 3s/step - loss: 0.4547 - accuracy: 0.9406 - val_loss: 0.4648 - val_accuracy: 0.9408
Epoch 4/5
1156/1156 [==============================] - 3079s 3s/step - loss: 0.4458 - accuracy: 0.9408 - val_loss: 0.4629 - val_accuracy: 0.9408
Epoch 5/5
322/322 [==============================] - 493s 2s/step - loss: 0.4637 - accuracy: 0.9408
Test Accuracy: 0.9407767057418823


In [6]:
# Prompt the user to enter their input
user_input = input("Enter the body text: ")

user_input_seq = Reg_tokenizer.texts_to_sequences([user_input])
user_input_pad = pad_sequences(user_input_seq, maxlen=maxlen, padding='post', truncating='post')

# Predict the title based on the user input
predicted_title_probs = Reg_model.predict(user_input_pad)

print("Predicted Title Probabilities:")

# Find the index of the word with the highest probability for each prediction
predicted_title_indices = predicted_title_probs.argmax(axis=2)

# Convert indices to words
predicted_titles = Reg_tokenizer.sequences_to_texts(predicted_title_indices)

print(predicted_titles)


1/1 [==============================] - 0s 50ms/step
Predicted Title Probabilities:
['play play store store']


In [16]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Get predictions
predictions = Reg_model.predict(X_test_pad)
y_pred = (predictions > 0.5).astype(int)

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

  8/322 [..............................] - ETA: 1:01:04